# Persistence

## Simplex def

In [617]:
class Simplex:
    def __init__(self, val, dim, vert):
        self.val = val
        self.dim = dim
        self.vert = vert
        
    def __eq__(self, simplex2):
        return self.vetices.__eq__(simplex2.verticles)
    

    def toString(self):
        return "val = {}; dim = {}; vert = {}".format(self.val, self.dim, "/".join(self.vert))

## Load the data

In [618]:
simplex_list = []

with open("filtrations/test_filtration2.txt", "r") as test_filtration:
    for line in test_filtration.readlines():        
        inputs = line.strip().split(" ")
        f, dim = float(inputs[0]), int(inputs[1])
        vertices = sorted(list(map(int, inputs[2:]))) # vertices are sorted so we can compare between list of vertices
        simplex_list.append(Simplex(f, dim, vertices))
        
simplex_list = sorted(simplex_list, key=lambda x:(x.val, x.dim)) # sort according to val and then to dim (to have a simplicial complex)


dic_simplex = {} # maps a simplex (as a sorted tuple of vertices) to its indentifier 
for i, s in enumerate(simplex_list):
    dic_simplex[tuple(s.vert)] = i
        

## Boundary matrix

In [619]:
def compute_boundary(simplex, dic_simplex):
    """return the identifiers of the boundaries of the simplex"""
    boundaries_id = []
    for i in range(simplex.dim + 1):
        new_vert = list(simplex.vert)
        del new_vert[i]
        if len(new_vert) > 0:
            boundaries_id.append(dic_simplex[tuple(sorted(new_vert))]) # new_vert must be sorted because we compare tuple (it's actually already sorted but we sort it for code clarity)
    return boundaries_id
        

In [620]:
# We define low(i) = the maximal non-zero element for the i-th simplex.

# dic_low[k] = list containing all the simplices i whose low(i) is in the k-th row, i.e. corresponds to the k-th simplex
# dic_boundaries[i] = list containing all the non-zero boundary simplices from the i-th simplex
dic_low, dic_boundaries = {},{} 
for i in range(len(simplex_list)):
    dic_boundaries[i] = [] # initialization
    dic_low[i] = []

for simplex in simplex_list:
    
    simplex_vert = tuple(sorted(simplex.vert))
    simplex_coord = dic_simplex[simplex_vert] #identifier of the simplex
    boundaries_id = compute_boundary(simplex, dic_simplex)
    if len(boundaries_id) > 0:
        dic_boundaries[simplex_coord] = boundaries_id
        low_i = max(boundaries_id)
        # Check if they're already another simplex with the same low(i)
        dic_low[low_i] = dic_low[low_i] + [simplex_coord] if (low_i in dic_low.keys()) else [simplex_coord] 
    for boundary_id in boundaries_id:
         #identifier of the boundary simplex
        dic_coord[(simplex_coord, boundary_id)] = 1
        


In [621]:
dic_boundaries

{0: [], 1: [], 2: [], 3: [1, 0], 4: [2, 1], 5: [2, 0], 6: [4, 5, 3]}

## Reduction

In [622]:
def sum_boundaries(boundaries1, boundaries2):
    """sum the list of boundaries of 2 simplex in Z/2Z
    Complexity could be improved"""
    res = []
    for b in boundaries1:
        if b not in boundaries2:
            res.append(b)
    for b in boundaries2:
        if b not in res and b not in boundaries1:
            res.append(b)
    return res
        

def gaussian_elim(dic_low, dic_boundaries):
    pivots = []
    for i in reversed(range(0, len(dic_boundaries.keys()))):
        if i in dic_low.keys():
            columns = sorted(dic_low[i])
            if len(columns) > 0:  # if there are columns to kill
                pivot_column = columns[0]
                pivots.append(pivot_column)
                for column in columns[1:]:
                    # sum in Z/2Z
                    dic_boundaries[column] = sum_boundaries(dic_boundaries[column], dic_boundaries[pivot_column])
                    if len(dic_boundaries[column]) > 0: 
                        # update the pivot
                        new_pivot = max(dic_boundaries[column])
                        if new_pivot in dic_low.keys():
                            dic_low[new_pivot].append(column)
                        else:
                            dic_low[new_pivot] = [column]
                            
    return pivots, dic_boundaries
            
pivots, dic_boundaries = gaussian_elim(dic_low, dic_boundaries)

In [623]:
dic_boundaries

{0: [], 1: [], 2: [], 3: [1, 0], 4: [2, 1], 5: [], 6: [4, 5, 3]}

## Bar code

In [624]:
def bar_codes(simplex_list, dic_boundaries):
    threshold = 0.05

    bar_codes = {}
    for i, s in enumerate(simplex_list):
        if len(dic_boundaries[i]) == 0: #create a cycle
            bar_codes[i] = None
        else: #kill a cycle (create a boundary)
            k = max(dic_boundaries[i]) # pivot
            if k in bar_codes.keys():
                bar_codes[k] = i

    for s1_id in bar_codes.keys():
        dim = simplex_list[s1_id].dim
        s2_id = bar_codes[s1_id]
        s1_val = simplex_list[s1_id].val
        if s2_id: # the cycle as been killed
            s2_val = simplex_list[s2_id].val
            if s2_val - s1_val > threshold:
                print(dim, s1_val, s2_val)
        else: # the cycle has never been killed
            s2_val = "inf"
            print(dim, s1_val, s2_val)
        
bar_codes(simplex_list, dic_boundaries)

0 1.0 inf
0 2.0 4.0
0 3.0 5.0
1 6.0 7.0
